## Import libraries

In [2]:
import glob
import os
import re
import c3d
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

### Find and compile C3D data

In [3]:
path = "D:\\Vicon Install\\FinchPerching\\" # change to correct folder location

In [5]:
all_c3d_filenames = sorted(glob.glob(path + r"*\*\*takeoff*.c3d"))
c3d_df = pd.DataFrame(columns=["Path","Bird","Date","Filename"])

for i, path in enumerate(all_c3d_filenames):
    bird = path.split("\\")[-3] # "<ring colour>R-<ring colour>L_<PIT tag>" -> bird identifier (folder name)
    date = path.split("\\")[-2] # "YYMMDD_day<bird age>" 
    filename = path.split("\\")[-1] # "YYMMDD_day<bird age>_<Trial number>_landing<number>.c3d"
    c3d_row = pd.DataFrame({"Path": path,"Bird": bird,"Date":date,"Filename":filename}, index=[0])
    c3d_df = pd.concat([c3d_df, c3d_row], axis=0, ignore_index=True)

### Load C3D data into pandas 

In [ ]:
rows_list = []

# iterate over all C3D files (in C3D dataframe)
for index, row in c3d_df.iterrows():    

    filepath = c3d_df.at[index, 'Path'] # get path to .c3d file
    bird = c3d_df.at[index, 'Bird'] # get bird ID
    bird_short = bird.split("_")[0] # get first part of bird ID (bird rings)
    date = c3d_df.at[index, 'Date'] # get date in format YYMMDD_day<age>
    age = date[12:14] # get age from date string (NEED CONSISTENT DATE FORMAT)
    if age == '10': # for birds aged 100 days, above method only gets first 2 digits (10) so need to change to 100
        age = '100'
    filename = c3d_df.at[index, 'Filename'] # get name of c3d file, format YYMMDD_day00_00_landing*.c3d
    match = re.search('takeoff(\d+)', filename) # search for number after 'landing' in filename string
    if match:
        takeoff = match.group(1) # return number if found
    else:
        takeoff = None # else return None

    # finally, if filename doesn't end with "<digit>.c3d", write the last bit of filename as a note
    if not filename[-5].isdigit(): 
        note = filename.split("_")[-1][:-4]
    else:
        note = None


    c3dfile = c3d.Reader(open(filepath, 'rb'))

    markers = [0,1,2,3,4,5,6,7,8,9]
    for marker in markers:
        for f, points, analog in c3dfile.read_frames():
            if marker < points.shape[0]:

                t = float(f)/120
                X = points[marker,0]
                Y = points[marker,1]
                Z = points[marker,2]

                if (X==0 or Y==0 or Z==0):
                    X, Y, Z = 'nan', 'nan', 'nan'

                data_row = {'Bird': bird_short, 
                            'Age': age, 
                            #'Category': category[c], 
                            'Takeoff': takeoff,
                            'Note': note,
                            'Marker': marker,
                            'Time': t,
                            'Frame': f,
                            'X': X,
                            'Y': Y,
                            'Z': Z,}
                # add new row to end of list of dictionaries
                rows_list.append(data_row)


# convert list of dictionaries to pandas DataFrame
trajectories_df = pd.DataFrame(rows_list)   

In [11]:
trajectories_df

,Bird,Age,Takeoff,Note,Marker,Time,Frame,X,Y,Z
0,blackR-blackL,20,1,None,0,0.008333,1,232.305817,-442.533295,1298.359863
1,blackR-blackL,20,1,None,0,0.016667,2,232.295502,-442.520752,1298.371094
2,blackR-blackL,20,1,None,0,0.025000,3,232.317398,-442.532898,1298.360474
3,blackR-blackL,20,1,None,0,0.033333,4,232.307358,-442.532135,1298.363403
4,blackR-blackL,20,1,None,0,0.041667,5,232.332626,-442.528687,1298.363892
...,...,...,...,...,...,...,...,...,...,...
1841515,yellowR-purpleL,100,12,None,8,3.966667,476,nan,nan,nan
1841516,yellowR-purpleL,100,12,None,8,3.975000,477,nan,nan,nan
1841517,yellowR-purpleL,100,12,None,8,3.983333,478,nan,nan,nan
1841518,yellowR-purpleL,100,12,None,8,3.991667,479,nan,nan,nan
